In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

2023-02-14 13:33:03.904027: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 13:33:05.993349: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/slurm-20.11.0/lib64:/opt/slurm/slurm-20.11.0/lib64:/opt/slurm/slurm-20.11.0/lib64:
2023-02-14 13:33:05.993440: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-14 13:33:18.217536: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dyna

In [3]:
tf.config.list_physical_devices

<function tensorflow.python.framework.config.list_physical_devices(device_type=None)>

### Try Enformer in TF

In [2]:
# Download targets from Basenji2 dataset 
# Cite: Kelley et al Cross-species regulatory sequence activity prediction. PLoS Comput. Biol. 16, e1008050 (2020).
targets_txt = 'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_human.txt'
df_targets = pd.read_csv(targets_txt, sep='\t')
df_targets.head(3)

,index,genome,identifier,file,clip,scale,sum_stat,description
0,0,0,ENCFF833POA,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:cerebellum male adult (27 years) and mal...
1,1,0,ENCFF110QGM,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:frontal cortex male adult (27 years) and...
2,2,0,ENCFF880MKD,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:chorion


In [43]:
pd.set_option('display.max_colwidth', -1)
df_targets.description

/gpfs/commons/home/tlin/miniconda3/envs/enformer/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


0       DNASE:cerebellum male adult (27 years) and male adult (35 years)    
1       DNASE:frontal cortex male adult (27 years) and male adult (35 years)
2       DNASE:chorion                                                       
3       DNASE:Ishikawa treated with 0.02% dimethyl sulfoxide for 1 hour     
4       DNASE:GM03348                                                       
            ...                                                             
5308    CAGE:epithelioid sarcoma cell line:HS-ES-2R                         
5309    CAGE:squamous cell lung carcinoma cell line:RERF-LC-AI              
5310    CAGE:gastric cancer cell line:GSS                                   
5311    CAGE:carcinoid cell line:NCI-H727                                   
5312    CAGE:lung adenocarcinoma, papillary cell line:NCI-H441              
Name: description, Length: 5313, dtype: object

In [3]:
CAGE = df_targets[df_targets.description.str.contains('CAGE')].drop(columns=['file','clip','scale'])

In [6]:
CAGE

,index,genome,identifier,sum_stat,description
4675,4675,0,CNhs10608,sum,CAGE:Clontech Human Universal Reference Total ...
4676,4676,0,CNhs10610,sum,CAGE:SABiosciences XpressRef Human Universal T...
4677,4677,0,CNhs10612,sum,CAGE:Universal RNA - Human Normal Tissues Bioc...
4678,4678,0,CNhs10615,sum,"CAGE:adipose tissue, adult, pool1"
4679,4679,0,CNhs10616,sum,"CAGE:bladder, adult, pool1"
...,...,...,...,...,...
5308,5308,0,CNhs14239,sum,CAGE:epithelioid sarcoma cell line:HS-ES-2R
5309,5309,0,CNhs14240,sum,CAGE:squamous cell lung carcinoma cell line:RE...
5310,5310,0,CNhs14241,sum,CAGE:gastric cancer cell line:GSS
5311,5311,0,CNhs14244,sum,CAGE:carcinoid cell line:NCI-H727


### Try Enformer in PyTorch

In [13]:
import torch
from enformer_pytorch import Enformer, seq_indices_to_one_hot

In [5]:
model = Enformer.from_hparams(
    dim = 1536,
    depth = 11,
    heads = 8,
    output_heads = dict(human = 5313, mouse = 1643),
    target_length = 896,
).cuda

In [11]:
import torch
seq = torch.randint(0,5,(196_608 // 2,)).cuda()
target = torch/rand(200,5313).cuda()
loss = model(
seq, head = 'human', target = target)

loss.backward()


## after much training

corr_coef = model(
seq, 
head = 'human', 
target = target, 
return_corr_coef = True)

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

## Building pre-train model

In [15]:
!pip install enformer-pytorch>=0.5

from enformer_pytorch import Enformer
enformer = Enformer.from_pretrained('EleutherAI/enformer-official-rough', from_tf=True)

OSError: EleutherAI/enformer-official-rough does not appear to have a file named pytorch_model.bin, tf_model.h5, model.ckpt or flax_model.msgpack.

In [14]:
! enformer-pytorch --version

/bin/bash: enformer-pytorch: command not found


In [9]:
from enformer_pytorch import Enformer

enformer = Enformer.from_pretrained('EleutherAI/enformer-official-rough', use_checkpointing = True)

# finetune enformer on a limited budget

OSError: Unable to load weights from pytorch checkpoint file for '/gpfs/commons/home/tlin/.cache/huggingface/hub/models--EleutherAI--enformer-official-rough/snapshots/affe5713ae9017460706a44108289b13c5fee16c/pytorch_model.bin' at '/gpfs/commons/home/tlin/.cache/huggingface/hub/models--EleutherAI--enformer-official-rough/snapshots/affe5713ae9017460706a44108289b13c5fee16c/pytorch_model.bin'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.

In [16]:
enformer = Enformer.from_pretrained('EleutherAI/enformer-official-rough', use_checkpointing = True)

In [9]:
import h5py

In [20]:
chr19 = h5py.File("/gpfs/commons/home/tlin/data/enformer/1000G.MAF_threshold=0.005.19.h5")

In [17]:
list(chr19.keys())

['SAD',
 'SAR',
 'alt',
 'chr',
 'pos',
 'ref',
 'snp',
 'target_ids',
 'target_labels']

In [22]:
data_matrix = np.random.uniform(-1, 1, size=(10, 3))

# Write data to HDF5
with h5py.File("file.hdf5", "w") as data_file:
    data_file.create_dataset("dataset_name", data=data_matrix)

In [23]:
data_matrix

array([[ 0.0221247 , -0.85046517,  0.49753035],
       [-0.02822134, -0.12902879,  0.28430481],
       [-0.31456732,  0.99252742, -0.53952311],
       [ 0.87378091,  0.26367463,  0.25386811],
       [ 0.32295486,  0.68736025,  0.94317663],
       [-0.43602191,  0.55567755,  0.16573492],
       [-0.92730231,  0.76182661, -0.58325799],
       [-0.30837322,  0.00253892, -0.45353027],
       [ 0.50880021,  0.52723648,  0.73025975],
       [ 0.41862606, -0.07916734,  0.25615949]])

In [ ]:
data

In [27]:
for key in chr19.keys():
    print(key) #Names of the root level object names in HDF5 file - can be groups or datasets.
    #print(type(chr19[key])) # get the object type: usually group or dataset
    

SAD
SAR
alt
chr
pos
ref
snp
target_ids
target_labels


In [33]:
ID = chr19["target_ids"]

In [52]:
ID[:].shape

(5313,)

In [46]:
%lsmagic


Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

In [50]:
%%bash
echo "test if this is working"
echo $PATH

test if this is working
/gpfs/commons/home/tlin/miniconda3/envs/enformer/bin:/nfs/sw/R/R-4.1.1/bin:/nfs/sw/R/R-4.1.1:/nfs/sw/gcc/gcc-11.2.0/bin:/nfs/sw/gcc/gcc-11.2.0:/opt/slurm/slurm-20.11.0/bin:/opt/slurm/pe2/slurm/centos7/bin:/opt/slurm/slurm-20.11.0/bin:/gpfs/commons/home/tlin/miniconda3/condabin:/gpfs/commons/home/tlin/.local/bin:/gpfs/commons/home/tlin/bin:/opt/slurm/slurm-20.11.0/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/usr/lpp/mmfs/bin:/opt/puppetlabs/bin:/usr/lpp/mmfs/bin:/usr/lpp/mmfs/bin


In [8]:
CAGE.index


Int64Index([4675, 4676, 4677, 4678, 4679, 4680, 4681, 4682, 4683, 4684,
            ...
            5303, 5304, 5305, 5306, 5307, 5308, 5309, 5310, 5311, 5312],
           dtype='int64', length=638)